In [1]:
import psycopg2
import streamlit as st
import pandas as pd

In [2]:
# 全局变量
# 基础表单的名称
Base_Table = "base_statistics"
# 日表单的名称
Day_Table = "daily_statistics"
# 周表单的名称
Week_Table = "weekly_statistics" 

In [3]:
# PostgreSQL 连接信息
def connect_to_postgresql():
    # 如果用docker，需要实际的容器 IP 地址替换这里
    DB_HOST = "localhost"
    # 数据库端口号，通常是默认的 5432
    DB_PORT = "5432"
    # 数据库名称
    DB_NAME = "filecoin"
    # 数据库用户名
    DB_USER = "postgres"
    # 数据库密码
    DB_PASS = "901205"
    
    try:
        # 连接到 PostgreSQL 数据库
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS
        )
        return conn
    except Exception as e:
        print(f"An error occurred while connecting to PostgreSQL: {str(e)}")
        return None

In [4]:
# 查询函数，这部分用于构建之后所有计算的基础dataframe
def query_data_from_postgresql(conn, table_name):
    try:
        # 构建 SQL 查询语句
        query_base = f"SELECT * FROM {table_name};"
        
        # 查询数据
        df = []
        df = pd.read_sql_query(query_base, conn)
        return df
    except Exception as e:
        print(f"An error occurred while querying data: {str(e)}")
        return None

In [5]:
# 计算日表函数，数据并返回一个包含新数据的 DataFrame 
def calculate_daily_statistics(df_base):
    df_day = pd.DataFrame()
    
    # 复制基础数据列到 df_day
    df_day['time'] = df_base['time']
    df_day['node_id'] = df_base['node_id']
    df_day['total_balance'] = df_base['total_balance']
    df_day['ava_balance'] = df_base['ava_balance']
    df_day['ini_pledge'] = df_base['ini_pledge']
    df_day['lock_balance'] = df_base['lock_balance']
    df_day['owner_balance'] = df_base['owner_balance']
    df_day['power'] = df_base['power']
    
    # 使用 groupby 按照 node_id 分组
    grouped = df_base.groupby('node_id')

    # 计算变化和比率列
    df_day['day_total_change'] = grouped['total_balance'].diff().round(2)
    df_day['day_total_rate'] = ((df_day['day_total_change'] / grouped['total_balance'].shift(1)) * 100).round(2)
    df_day['day_ava_change'] = grouped['ava_balance'].diff().round(2)
    df_day['day_ava_rate'] = ((df_day['day_ava_change'] / grouped['ava_balance'].shift(1)) * 100).round(2)
    df_day['day_ini_change'] = grouped['ini_pledge'].diff().round(2)
    df_day['day_ini_rate'] = ((df_day['day_ini_change'] / grouped['ini_pledge'].shift(1)) * 100).round(2)
    df_day['day_lock_change'] = grouped['lock_balance'].diff().round(2)
    df_day['day_lock_rate'] = ((df_day['day_lock_change'] / grouped['lock_balance'].shift(1)) * 100).round(2)
    df_day['day_owner_change'] = grouped['owner_balance'].diff().round(2)
    df_day['day_owner_rate'] = ((df_day['day_owner_change'] / grouped['owner_balance'].shift(1)) * 100).round(2)
    df_day['day_power_change'] = grouped['power'].diff().round(2)
    df_day['day_power_rate'] = ((df_day['day_power_change'] / grouped['power'].shift(1)) * 100).round(2)

    # 计算 day_luck（这个特殊），注意此处先计算，后分组
    df_day['day_luck'] = (df_day['day_total_change'] / df_day['power'] / df_base['luck']).round(2)

    # 再次使用 groupby 按照 node_id 分组
    grouped_day = df_day.groupby('node_id')
    df_day['day_luck_change'] = grouped_day['day_luck'].diff().round(2)
    df_day['day_luck_rate'] = ((df_day['day_luck_change'] / df_day['day_luck'].shift(1)) * 100).round(2)

    # 移除第一行 NaN 值
    df_day = df_day.dropna()
    
    return df_day



In [6]:
# 更新 daily_statistics 表的函数
def update_daily_statistics(conn, df_day):
    try:
        cursor = conn.cursor()
        for index, row in df_day.iterrows():
            cursor.execute("""
                INSERT INTO daily_statistics
                (time, node_id, total_balance, day_total_change, day_total_rate,
                ava_balance, day_ava_change, day_ava_rate, ini_pledge, day_ini_change,
                day_ini_rate, lock_balance, day_lock_change, day_lock_rate, owner_balance,
                day_owner_change, day_owner_rate, power, day_power_change, day_power_rate,
                day_luck, day_luck_change, day_luck_rate)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (row['time'], row['node_id'], row['total_balance'], row['day_total_change'], row['day_total_rate'],
                    row['ava_balance'], row['day_ava_change'], row['day_ava_rate'], row['ini_pledge'], row['day_ini_change'],
                    row['day_ini_rate'], row['lock_balance'], row['day_lock_change'], row['day_lock_rate'], row['owner_balance'],
                    row['day_owner_change'], row['day_owner_rate'], row['power'], row['day_power_change'], row['day_power_rate'],
                    row['day_luck'], row['day_luck_change'], row['day_luck_rate']))
        
        conn.commit()
        print("Data successfully updated in daily_statistics.")
    except Exception as e:
        print(f"An error occurred while updating daily statistics: {str(e)}")


In [ ]:
# 主函数开始运行
# def main():

In [7]:
# 连接到 PostgreSQL
conn = connect_to_postgresql()
print (conn)

<connection object at 0x00000230C396D030; dsn: 'user=postgres password=xxx dbname=filecoin host=localhost port=5432', closed: 0>


In [8]:
# 生成基础表单的df数据
if conn is not None:
    # 指定要查询的表名
    base_table_name = Base_Table

    # 执行查询函数，生成基础数据df表单
    df_base = query_data_from_postgresql(conn, base_table_name)
    print(df_base)

An error occurred while querying data: Execution failed on sql 'SELECT * FROM base_statistics;': 错误:  关系 "base_statistics" 不存在
LINE 1: SELECT * FROM base_statistics;
                      ^

None


C:\Users\gx\AppData\Local\Temp\ipykernel_13016\2009684446.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_base, conn)


In [ ]:
if df_base is not None:
    # 执行日计算函数，生成日表单
    df_day = calculate_daily_statistics(df_base)
    print(df_day)

In [ ]:
# 调用函数更新 daily_statistics 表
update_daily_statistics(conn, df_day)            
# 查看更新后的 daily_statistics 表
day_table_name = Day_Table
df_day_ = query_data_from_postgresql(conn, day_table_name)

In [ ]:
# 关闭数据库连接
conn.close()